In [5]:
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish

from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os 
import schedule

import pandas as pd
import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import style

from copy import deepcopy
from IPython.display import clear_output

In [6]:
def getRankData():

    t = time.localtime(time.time())
    year = t.tm_year
    month = t.tm_mon
    day = t.tm_mday
    hour = t.tm_hour
    rlist = []

    client=mqtt.Client()
    client.connect("120.126.136.17",1883,60)
    topic = ""
    mongo = MongoClient('120.126.136.17', 27017)
    db = mongo['Tracker']


    collection = db['User_Info']
    friendCol = collection.find()

    for x in friendCol:
        name = x['username']
        if name!= 'pitest' and name!= 'jack' and name!= 'manager':
            rlist.append(name)
    rsize=len(rlist)
    
    collection = db['rank']
    
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour})

    #排名
    for x in data:
        topic = x['user']
        print(topic)
        rank =1
        pre=99999
        pro=-99999
        for name in rlist:
            if topic != name:
                if x[name]>0 and x[name]<pre:
                    rank=rank+1
                    pre=x[name]                 
                if x[name]<0 and x[name]>pro:
                    pro=x[name]
        if rank==1 and pre==99999:
            rank_mes_top = str(rank)+"/"+str(rsize)
            rank_mes_top = '%-7s' % rank_mes_top
            rank_mes_top = "排名 "+ rank_mes_top
            rank_mes_bot = "第"+str(rank)+"名,好棒棒"
            rank_mes=rank_mes_top+rank_mes_bot
            print(rank_mes)
            client.publish(topic,rank_mes) 
        elif pro==-99999:
            rank_mes_top = str(rank)+"/"+str(rsize)
            rank_mes_top = '%-7s' % rank_mes_top
            rank_mes_top = "排名 "+ rank_mes_top
            rank_mes_bot = "第"+str(rank)+"名,太混囉"
            rank_mes=rank_mes_top+rank_mes_bot
            print(rank_mes)
            client.publish(topic,rank_mes)
        else:
            rank_mes_top = str(rank)+"/"+str(rsize)
            rank_mes_top = '%-7s' % rank_mes_top
            rank_mes_top = "排名 "+ rank_mes_top
            rank_mes_bot = "第"+str(rank)+"名,再加油"
            rank_mes=rank_mes_top+rank_mes_bot
            print(rank_mes)
            client.publish(topic,rank_mes)
        time.sleep(0.01)
            
    time.sleep(5)
    print("======================")
    
    #前一名
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour})
    
    for x in data:
        topic = x['user']
        print(topic)
        rank =1
        pre=99999
        pro=-99999
        for name in rlist:
            if topic != name:
                if x[name]>0 and x[name]<pre:
                    rank=rank+1
                    pre=x[name]                 
                if x[name]<0 and x[name]>pro:
                    pro=x[name]
        if rank==1 and pre==99999:
            rank_mes = "保持冠軍喔!"
            print(rank_mes)
            client.publish(topic,rank_mes) 
        else:
            rank_mes_top = "與前一名落差"
            rank_mes_bot = str(int(pre))+"步"
            rank_mes=rank_mes_top+rank_mes_bot
            print(rank_mes)
            client.publish(topic,rank_mes)
            
    time.sleep(5)
    print("======================")
    
    #後一名
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour})
    
    for x in data:
        topic = x['user']
        print(topic)
        rank =1
        pre=99999
        pro=-99999
        for name in rlist:
            if topic != name:
                if x[name]>0 and x[name]<pre:
                    rank=rank+1
                    pre=x[name]                 
                if x[name]<0 and x[name]>pro:
                    pro=x[name]
        if pro==-99999:
            rank_mes = "吊車尾加把勁"
            print(rank_mes)
            client.publish(topic,rank_mes) 
        else:
            rank_mes_top = "與後一名落差"
            rank_mes_bot = str(abs(int(pro)))+"步"
            rank_mes=rank_mes_top+rank_mes_bot
            print(rank_mes)
            client.publish(topic,rank_mes)
            
    mongo.close()
    client.disconnect()

In [12]:
def getHourData():

    t = time.localtime(time.time())
    year = t.tm_year
    month = t.tm_mon
    day = t.tm_mday
    hour = t.tm_hour

    client=mqtt.Client()
    client.connect("120.126.136.17",1883,60)
    topic = ""
    

    mongo = MongoClient('120.126.136.17', 27017)
    db = mongo['Tracker']
    collection = db['prediction']
    
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour,'type':'hourly'})
    
    for x in data:
        topic = x['user']
        print(topic)
        top = '步數:%5s步' % int(x['totalsteps'])
        bot = '目標:%5s步' % x['prediction']
        mes = top+bot
        print(mes)
        client.publish(topic,mes)
        time.sleep(0.01)
    
    mongo.close()
    client.disconnect()

In [15]:
getHourData()

leo
步數:  685步目標:    6步
james
步數: 7490步目標:  796步
shuan
步數:  916步目標:    0步
t
步數:    0步目標:    0步
jeng
步數: 1271步目標:  224步


In [16]:
# getHourData()
getRankData()
# realtimetotalstep()

t
排名 3/5    第3名,太混囉
jeng
排名 3/5    第3名,太混囉
leo
排名 3/5    第3名,太混囉
james
排名 1/5    第1名,好棒棒
shuan
排名 2/5    第2名,再加油
t
與前一名落差259步
jeng
與前一名落差259步
leo
與前一名落差259步
james
保持冠軍喔!
shuan
與前一名落差6742步
t
吊車尾加把勁
jeng
吊車尾加把勁
leo
吊車尾加把勁
james
與後一名落差6742步
shuan
與後一名落差259步
